Clustering simulated sequences using TMRCA using an array job (not working)

Name: CQS21 
FYP 2024

In [5]:
# installing packages to my python environment
# NOTE: my pip is v22.0.4 but I cant seem to update to the newest 24.0

#!pip install zarr
#!pip install scipy
#!pip install scikit-allel
#!pip install matplotlib
#!pip install tqdm
#!pip install dask
#!pip install seaborn
#!pip install tskit
#!pip install msprime
#!pip install pyslim

##conda install? matplotlib 
# do not do pip after using conda 

global genome_length
genome_length =70000
global mutation
mutation = int((genome_length+1)/2) 
global threshold
threshold = 0.87  #setting threshold to calculate trough points in each haplotype
global points
points=280
global window 
window=600
global sample_size
sample_size = 100
global no_haplotypes
no_haplotypes = sample_size*2

import os
global array_index 
array_index = int(os.environ['PBS_ARRAY_INDEX']) # HPC ppl suggested using os module, gets it as a str, need to int() it
#instead of using sys.argv[1] which takes the first argument passed
print (array_index)

global seed
seed = int(os.environ['seed'])
print (seed)

KeyError: 'PBS_ARRAY_INDEX'

In [6]:
# Import modules
import numpy as np
import zarr
import allel   
#idk why but my allel doesnt seem to load?? even though ive definitely installed it as scikit-allel
import scipy.cluster.hierarchy as sch
import scipy.spatial
import matplotlib
import matplotlib.pyplot as plt
import scipy.signal
from scipy.ndimage import gaussian_filter1d
from numpy.lib.stride_tricks import sliding_window_view
from tqdm import tqdm
import dask
from dask.delayed import delayed
from dask.base import compute
from itertools import combinations
import seaborn as sns
#import tskit
import pandas as pd

SEE HPC STEPS.ipynb

Step 1. Generate burn in simulations establishing nucleotide diversity across a range of parameters (redo theo's work but w updated vgsc parameters)

Step 2. Conduct 27 variations of partial soft sweep simulations and output vcf files using the HPC facility. Complete x number of repeats.

Size of population, N: 100, 1000, 10000

Mutation rate, μ : (0.1/4N), (1/4N), (5/4N)

Recombination rate, r : (0.1/4N), (1/4N), (5/4N)

In [ ]:
# checking .tree outputs
#import tskit
#tskit.load(file="C:/CHEYANNE STUFF/ICL Biology/Year 3/Final year project/burnin_no.19_early.trees")
#tskit.load(file="C:/CHEYANNE STUFF/ICL Biology/Year 3/Final year project/burnin_no.19_early_10.trees")


# install pyslim !!! issue here involving msprime? pyslim cannot be imported/downloaded
#import pyslim

# read into pyslim and calculate nucleotide diversity and compare against expected nucleotide diversity (theta)
# check that it exceeds 10% and not more than 20%


Step 3. Read in VCF Files from soft sweep simulations, extract haplotypes using scikit_allel

Working with 27 simulation variants and repeats of each variant (100?)

but why only look at haplotypes of first 200 individuals? and why calculate sample frequency over specifically 400 haplotypes? is it linked to the 200 individuals = 400 genomes?


In [ ]:
# load example vcf from SLIM
#with open('example.vcf', mode='r') as vcf:\

    #print(vcf.read())

#OUT: 10000 10000 GV 20
##fileformat=VCFv4.2
##fileDate=20160613
##source=SLiM
##slimGenomePedigreeIDs=4999898,4999899,...
##INFO=<ID=MID,Number=1,Type=Integer,Description="Mutation ID in SLiM">
##INFO=<ID=S,Number=1,Type=Float,Description="Selection Coefficient">
##INFO=<ID=DOM,Number=1,Type=Float,Description="Dominance">
##INFO=<ID=PO,Number=1,Type=Integer,Description="Population of Origin">
##INFO=<ID=TO,Number=1,Type=Integer,Description="Tick of Origin">
##INFO=<ID=MT,Number=1,Type=Integer,Description="Mutation Type">
##INFO=<ID=AC,Number=1,Type=Integer,Description="Allele Count">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Total Depth">
##INFO=<ID=MULTIALLELIC,Number=0,Type=Flag,Description="Multiallelic">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
#CHROM POS ID REF ALT QUAL FILTER INFO FORMAT i0 i1 i2 i3 i4 i5...
1 1309 . A T 1000 PASS
MID=987550;S=0;DOM=0.5;PO=2;TO=9404;MT=1;AC=11;DP=1000 GT 0|1 0|0 0|1...
1 5995 . A T 1000 PASS
MID=987764;S=0;DOM=0.5;PO=1

In [8]:
#prev student: Like Hamming distance code, this was also taken from Anushka Thawani. Adaptations were made to this on the 
#high-performance computer using shell script, but this could not be represented.
def convert(file):
    '''
    This function extracts haplotypes sequences from a vcf file 
    Adapted from: http://alimanfoo.github.io/2018/04/09/selecting-variants.html 
    
    Arguments:
        file: name of vcf file (from SLiM soft sweep simulation)
        
    Returns:
        ht: haplotype sequences for 200 individuals
        samp_freq: frequency of sweep mutation in sample
        cols: used to color dendrogram
        pos: 

    '''
    
    v = file + '.vcf'
    z = file + '.zarr'
    slim_sim_data = allel.read_vcf(v, fields='*')
    allel.vcf_to_zarr(v, z, fields='*', overwrite=True)
    data = zarr.open_group(z, mode='r')
    
    # Stores the ID and genomic position of each variant 
    pos = allel.SortedIndex(data['variants/POS'])             #KEEPS THROWING KEY ERROR meaning pos start and stop is the same! Why!
    
    # Extract genotypes for the first 200 individuals and convert to haplotypes
    gt = data['calldata/GT'][:,0:200] 
    ht = allel.GenotypeArray(gt).to_haplotypes()
    
    mutation = int((genome_length+1)/2) # position of sweep mutation
    
    
    # Output the frequency of the sweep mutation in the sample
    contains_sweep = pos.locate_range(mutation,mutation) #finds index of sweep mutation in the array
    sweep = ht[contains_sweep]                           # saves the haplotypes containing the sweep in the variable sweep
    sweep = np.sum(sweep, axis =0)                       #sums up mutation occurences in each haplotypes
    
    samp_freq = np.sum(sweep)/200  # finds freq in the entire sample of 100 individuals, 200 haplotypes
    
    
    # This dictionary is used later to color the dendrogram branches according to whether or not the 
    # corresponding sequence contains the sweep mutation
    cols = {}
    for i in range(200):
        if sweep[i]:
            cols[i] = "#FF0000" 
        else:
            cols[i] = "#808080"
    
    return ht, pos, samp_freq, cols, sweep

Step 4. Calculate Hij (S or homozygosity) for all pairs of haplotypes
make a function 
homozygosity = (no. of SNPs/ length of window, L)

In [9]:
def sliding_homozygosity(ht, pos):
    '''
    This function calculates the sliding homozygosity for all pairs of haplotypes.
    
    Arguments:
    window: length of sliding window
    len_genome : length of genome 
    ht : vector?  of haplotype sequences from convert() function in previous codeblock
    pos: position of variants from convert() function in previous codeblock

    
    Returns:
    homozygosities:  homozygosity of all haplotypes in ht in a array(?)
    '''
    # Make empty vectors
    homozygosities = []
    
    #iterate through all nucleotides in the genome
    for x in range(0, genome_length):
        #define nt ranges 
        start  = x
        end = x + window
        # locate the position of region around a variant Nt
        region = pos.locate_range(start,end) 
        haplotype_region = ht [region]

        #use allel.pairwise distance
        pairwise_dist = allel.pairwise_distance(haplotype_region, metric = 'hamming' , chunked=True, blen=None) #can i chunk = true to speed up computation? 
        homozygosities = pairwise_dist/window

    return homozygosities



## TO DO streamlining: previous students code say to add a if condition (region =/= prev region to speed up code)

Step 5. Calculate Lij (shared haplotype length) from Hij by finding width at half maximum homozygosity

In [10]:
#half_max_homozygosity = (max(homozygosities) + min(homozygosities))/2

#Defining function to find troughs, to be used in calculating Lij in another function
def finding_troughs(smooth, pos):
    '''
    This function finds troughs for a pair of haplotype sequences. 
    Note: threshold set to 0.87
    
    Arguments:
        sliding homozygosity: smoothed sliding window homozygosity for all pairs of sequences
        sweeploc: position of sweep mutation in genome (same as previous function)
        
    Returns:
        lower: position of breakpoint left of the sweep site
        upper: position of breakpoint right of the sweep site
        SHL: shared haplotype length
    '''
    global threshold
    threshold = 0.87  #setting threshold to calculate trough points in each haplotype

    #finding troughs
    troughs = scipy.signal.find_peaks(-smooth) #- sign inverts the graph so the 'peaks' are our troughs
    troughs = troughs[0]     # Indexes of all troughs
    troughs = troughs[smooth[troughs] < threshold]   # Extract troughs where homozygosity<threshold
    
    #finds the peaks
    peaks = scipy.signal.find_peaks(smooth)
    peaks = peaks[0]  #indexes peaks
    
    # Find positions of troughs flanking sweep site
    bp = np.searchsorted(troughs,pos)  #search sorted finds index of position where mutation should be inserted in order to maintain the same order
    lower = troughs[bp - 1] #index of sweep site -1
    upper = troughs[bp]  #index of sweep site
    
    # Find the average peak position around the sweep site
    highest = peaks[(peaks >= lower) & (peaks <= upper)]
    if highest.size != 0:
        highest = np.mean(highest)
    else: 
        highest = (lower+upper)/2
    
    
    lower = (lower+highest)/2
    upper = (upper+highest)/2

    SHL = upper - lower
    
    return int(lower), int(upper), SHL



In [11]:
## function using each haplotype pair to return SHL and find lower and upper limits of SHL
# uses finding_troughs()
def find_breakpoint(haplotype_pair):
    '''
    For a pair of sequences, this function smoothes the sliding homozygosity and returns the SHL
    Arguments:haplotype_pair
        haplotype_pair: a pair of haplotype sequences
        
    Returns:
        lower: position of breakpoint left of the sweep site
        upper: position of breakpoint right of the sweep site
        SHL: shared haplotype length
    '''
    
    mutation_pos = mutation 
    smooth = gaussian_filter1d(haplotype_pair, points)
    try:
        lower, upper, SHL = finding_troughs(smooth, mutation_pos)
    except IndexError:
        lower = -1.3
        upper = -1.3
        SHL = -1.3
        
    return lower, upper, SHL

Step 6. Calculate Tij (Time to common ancestor) from Lij and Kij (which is no. of SNPs) on each shared length (haplotype)

𝜏_𝑖𝑗=(𝑘_𝑖𝑗+1)/(2ℓ_𝑖𝑗 (𝑟+𝜇))

In [ ]:
# Calculating Kij (No. of SNPs in each SHL)

def calculating_kij(gts,ht,result_find_breakpoint,pos):
    '''
    This function finds the number of SNPs over the shared haplotype length for all pairs of haplotype sequences
    
    Arguments:
        gts: number of haplotype pairs
        ht: haplotype sequnces
        result_find_breakpoint: output from find_breakpoint function
        
    Returns:
        diffs: number of SNPs for all pairs of haplotype sequences
        
    '''
    pairwise = []
    for combo in combinations(list(range(0,no_haplotypes)), 2): 
        pairwise.append(combo)

    diffs = np.empty(shape=(gts),dtype=np.float32)
    for i in range(gts):
        pair = ht[:,pairwise[i]]
        try:
            start = result_find_breakpoint[i,1]
            stop = result_find_breakpoint[i,2]

            window_pos = pos.locate_range(start, stop)
            window = pair[window_pos]

            d = allel.pairwise_distance(window, metric = "hamming")

            diffs[i]=d 

        except KeyError:
            diffs[i]=-1.3 
    
    return diffs

In [14]:
#Calculating Tij, Time to Common Ancestor (TMRCA) using mutation rate and number of SNPs
# 𝜏_𝑖𝑗=(𝑘_𝑖𝑗+1)/(2ℓ_𝑖𝑗 (𝑟+𝜇))
#for array number x:
    #read in vcf file
    # read in population size, mu, r from population parameters csv file


def calculating_Tij(array_index, seed, cutoff):
    '''
    This function calculates Tij, Time to Common Ancestor (TMRCA) using mutation rate and number of SNPs for each haplotype pair.
    It uses the convert() to convert files from vcf 
    It uses the sliding_heterozygosity() to calculate heterozygosity in the window for all pairs of haplotypes

    
    Arguments:
        array_index: array index or combination number of simulation
        seed: seed number of simulation
        cutoff: 40 % allele freq for rdl and 80% for vgsc

        
        genome_length: length of genome (in SLiM simulation)             ##can loop?
        ht : haplotypes (what variable structure?)

        window: length of sliding window
        threshold: threshold above which troughs are ignored
        points: number of points to use for 1D-gaussian filter (see scipy documentation)
        
    Returns:
        Tij: Time to Common Ancestor (TMRCA)
  
    '''
    import numpy as np
    
    
    ## read in population size, mu, r from population parameters csv
    parameters = pd.read_csv('C:/CHEYANNE STUFF/ICL Biology/Year 3/Final year project/Estimating-effective-population-size-from-resistance-loci/simulations/parameter_combinations.csv')
    index = array_index +1
    global pop_size
    pop_size = parameters.iloc[index]['N']
    global mu
    mu = parameters.iloc[index]['Mutation Rate']
    global r
    r = parameters.iloc[index]['Recombination Rate']


    # for array number:
        #read in vcf file
    file = "C:/CHEYANNE STUFF/ICL Biology/Year 3/Final year project/Estimating-effective-population-size-from-resistance-loci/Post Simulation/" + str(array_index) + "_"+ str(seed) +"_"+ str(cutoff)  #".vcf" added by convert()
    # Extract haplotype sequences from .vcf file
    ht, pos, samp_freq, cols, sweep = convert(file)

    
    # Calculate sliding homozygosity for all pairs of haplotype sequences
    gts = int((no_haplotypes*(no_haplotypes-1))/2)
    homozygosities = sliding_homozygosity(ht, pos)

    
    # Find SHL for all pairs of haplotype sequences 
    hom_dask = dask.array.from_array(homozygosities, chunks=(genome_length,1)) # type: ignore # creates a dask array
    homozygosities = []
    results = dask.array.apply_along_axis(find_breakpoint, 0, hom_dask) # type: ignore #applies find_breakpoint() along the array
    results_computed = results.compute()

    # Manipulating the dataframe to make it easier to process
    results_computed = np.transpose(results_computed)
    index = np.asarray(range(0,gts))
    index = np.expand_dims(index, axis=0)
    results_computed_1 = np.concatenate((index.T, results_computed), axis=1)
    
    
    # Calculate the TMRCA from the SHLs and number of SNPs
    recombination_rate = r/(2*pop_size)
    mu = mu/(100*pop_size)
    shls = results_computed_1[:,3]   # SHLs for all pairs of haplotype sequences 
    shls[shls<=0] = genome_length
    diffs = calculating_kij(gts, ht, results_computed_1, pos)  # SNPs for all pairs of haplotype sequences 
    Tij = (1+(diffs*shls))/(2*shls*(recombination_rate + mu)) # TMRCA metric for all pairs of haplotype sequences 

    
    # Remove negative and non-integer TMRCA values
    impute = np.nanmean(Tij)        #impute is the mean of SNP array without any NAN values
    x = np.isfinite(Tij)            #x is a boolean mask array of only finite values (cannot be infinite or NAN)
    for i in np.where(x == 0)[0]:   #for all indices where there is a non-finite number:
        Tij[i] = impute             #replace NaN with inpute value
    Tij[Tij<=0] = impute            # replace all negative numbers wih inpute

    return Tij, cols, pop_size, mu, r
    

Step 7. Plotting TMRCA dendrogram 
- prev student ran into problems here with generating the dendrogram coloured tips
- Untested to see if I have resolved previous student problem


In [15]:

def analysis(array_index, seed, cutoff): 

    '''
    This function plots a dendrogram and colours red the tips that have the sweep mutations. 
    It uses calculating_Tij().
    Only the output vcf from the SLIM simulation is input.
    global variables genome_length, window, threshold, points being used

    Arguments:
    array_index: array index or combination number of simulation
    seed: seed number of simulation
    cutoff: 40 % allele freq for rdl and 80% for vgsc
    
    global variables/ variables from sub functions:
    window: length of sliding window
    threshold: threshold above which troughs are ignored
    points: number of points to use for 1D-gaussian filter (see scipy documentation)
    cols: colours haplotype branches red if they have sweep mutation. From convert().



    Returns:
    output dendrogram in pdf
    '''
    #all convert(), etc etc to end up with tij
    Tij, cols, pop_size, mu, r = calculating_Tij(array_index, seed, cutoff)
    
    # Hierachical Clustering, store in Z
    Z = sch.linkage(Tij, method = 'average') #why do we use the Farthest Point Algorithm? changed to average (UPGMA) , check after
    

    ## Plot dendrogram without colouring branches
    # updating matplotlib font settings
    matplotlib.rcParams.update({'font.size': 24})
    fig = plt.figure(figsize=(30, 12))
    gs = matplotlib.gridspec.GridSpec(2, 1, hspace=0.1, wspace=1, height_ratios=(1,1)) # type: ignore

    ax_dend = fig.add_subplot(gs[0, 0])
    sns.despine(ax=ax_dend, offset=5, bottom=True, top=True)
    dd = sch.dendrogram(Z,color_threshold=0,above_threshold_color='#808080',ax=ax_dend) # if above colour threshold, set colour to grey 

    ls = []
    for leaf, leaf_color in zip(plt.gca().get_xticklabels(), dd["leaves_color_list"]):   #leaves_color_list is A list of color names. The k’th element represents the color of the k’th leaf.
        leaf.set_color(cols[int(leaf.get_text())])
        ls.append(int(leaf.get_text()))

    ax_dend.set_ylabel('Haplotype age/generations',fontsize=24)
    ax_dend.set_title('Haplotype clusters',fontsize=24)



    # Plot dendrogram and colour branches
    ax_dend_2 = fig.add_subplot(gs[1, 0])
    
    dflt_col = "#808080"
    
    link_cols = {}
    for i, i12 in enumerate(Z[:,:2].astype(int)):
        c1, c2 = (link_cols[x] if x > len(Z) else cols[x] for x in i12)
        link_cols[i+1+len(Z)] = c1 if c1 == c2 else dflt_col

    sns.despine(ax=ax_dend_2, offset=5, bottom=True, top=True)
    dd = sch.dendrogram(Z,link_color_func=lambda x: link_cols[x], ax=ax_dend_2)

    ls = []
    for leaf, leaf_color in zip(plt.gca().get_xticklabels(), dd["leaves_color_list"]):
        leaf.set_color(cols[int(leaf.get_text())])
        ls.append(int(leaf.get_text()))

    ax_dend_2.set_ylabel('Haplotype age/generations',fontsize=24)
    
    
    # Save dendrogram
    output_directory = '/dendrograms/' 
    output = output_directory + 'dendrogram_' + str(array_index) + "_"+ str(seed) +"_"+ str(cutoff) +'.pdf' 
    print (output)
    plt.savefig(output)  

In [16]:
#test 

#run analysis
analysis(array_index, seed, 40)
#analysis(array_index, seed, 80)

KeyError: (35001, 35001)

Step 4. Estimate nhat by counting number of origins in dendrogram
I think has to be done manually... have not solved how to get it done with python

do it for each and then record in the results.csv

# count number of origins (haplotypes) for all files and save in a list

def sum_origins (number_of_files, files):
    '''
    takes the number of simulations, the .vcf files and 
    calculates the number of origins per simulation and saves it in a list.


    returns:

    '''
    # search if can use sch.dendrogram to get number of haplotype origins 
    origins = []
    for i in range (number_of_files):
        read.file ( file + i + .TREES) as file: 

            #if _________ =___________:
                #calculate  number of origins as file_origins
        #attach file_origins to origins
    return print (origins)

    

Step 5. Calculate Ne hat by maximum likelihood estimation



#plot all estimated numbers of origins

# find a function that plots and finds maximum likelihood and the 95% confidence intervals
# calculating confidence interval by 
# 1. log ne hat and find maximum which should be the maximum of the graph as well
# average(origins) ?

import pandas as pd

def NeHat():
    '''
    This function plots the number of independent gene loci/origins (L) against population size (N). It also tells us NeHat (the mean), and the 95% confidence intervals.
    Do we just do the confidence intervals per N? 
    
    Arguments:
        csv_file_path: filepath of the results.csv used to manually record down the number of origins per simulation
        
    Returns:
        NeLow: lower point of 95% confidence intervals of NeHat being correct
        NeHigh: higher point of 95% confidence intervals of NeHat being correct
        NeHat: NeHat (at ln(Lmax))

    ''' 
    #load modules
    import pandas as pd
    import numpy as np
    from scipy.stats import norm
    import matplotlib.pyplot as plt
    import scipy

    #load csv
    origins = pd.read_csv('simulations/results.csv')


    # log the heterozygosity (ln(L))
    origins["ln(L)"] = np.log(origins['origins'])
    
    # At max ln(L), is Ne Hat
    #Lmax = origins['origins'].max()
    #ln_Lmax = np.log(Lmax) 
    
    # at [ln(Lmax)-2], Nelow and NeHigh are the confidence intervals
    #confidence_cutoff = ln_Lmax -2 
    #NeLow =       ##how to find best fit line and intersection?

    
    # load input variables from a file
    data =origins[['N','ln(L)']].copy()

    #subset by N and then loop through each 
    size_100 = data[(data["N"] == 100)]
    size_1000 = data[(data["N"] == 1000)]
    size_10000 = data[(data["N"] == 10000)]
    print (size_100)                                            
    
    # Fit a normal distribution to the data:
    mean, std = norm.fit(size_100)
    NeHat = mean #NeHat is the mean

    # get confidence intervals
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=1) #gets Confidence interval with equal areas around the mean
    #print(ci)

    # Plot 
    plt.plot(size_100['N'], size_100['ln(L)'] )
    plt.show()

    # Plot the PDF.
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 1)
    p = norm.pdf(x, mean, std)
    plt.plot(x, p, 'k', linewidth=2)
    title = "Fit results: mu = %.2f,  std = %.2f" % (mean, std)
    # confidence interval left line
    one_x12, one_y12 = [ci[0],ci[0]], [0, 20]
    # confidence interval right line
    two_x12, two_y12 = [ci[1],ci[1]], [0, 20]
    plt.plot(one_x12, one_y12, two_x12, two_y12, marker = 'o')
    plt.title(title)

    plt.show()

    NeLow = ci[0]
    NeHigh = ci[1]
    print ('NeLow, NeHigh:' + str(NeLow) + "," + str(NeHigh))

    # plot to look at confidence intervals, NeHat
    #Lmax = max (df['Heterozygosity'])
    #ln_Lmax = np.log(Lmax) 
    #plt.plot('cycle','Heterozygosity', data = df)
    #plt.axhline(y =ln_Lmax, color='r', linestyle='-') #ln_Lmax
    #plt.axhline(y=confidence_cutoff, color='r', linestyle='-') #confidence interval cutoff
    #plt.show()


    return NeLow, NeHigh, NeHat

NeHat()